In [1]:
import os

# Force CPU backend on Apple Silicon to avoid Metal issues
os.environ["JAX_PLATFORMS"] = "cpu"

# Disable LaTeX rendering in matplotlib
import matplotlib.pyplot as plt

plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "sans-serif"

import pickle
from pathlib import Path

import jax.numpy as jnp
from jax import random
from sbijax import NLE, plot_loss_profile
from sbijax.nn import make_maf

from vr_foraging_sbi_ddm.simulator import PatchForagingDDM_JAX, create_prior
from vr_foraging_sbi_ddm.snle.snle_inference_jax import infer_parameters_snle, train_snle
from vr_foraging_sbi_ddm.snle.snle_utils_jax import extract_samples, get_model_directory, pairplot, plot_real_synth_hist
from vr_foraging_sbi_ddm.validation import plot_recovery_scatter, run_sbc_evaluation, validate_parameter_recovery

c:\git\AllenNeuralDynamics\sbi-ddm\.venv\Lib\site-packages\arviz\__init__.py:50: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


In [ ]:
# ============================================================================
# Configuration
# ============================================================================

CONFIG = {
    "base_path": Path("/Users/laura.driscoll/Documents/data/VR foraging/vr_foraging_data"),
    "base_output_dir": Path("/Users/laura.driscoll/Documents/code/sbi_results"),
    "filename": [],
    "window_size": 100,
    "n_feat": 37,
    "odor_types": ["Methyl_Butyrate", "Alpha_pinene"],
    "odor_display_names": {"Methyl_Butyrate": "Methyl Butyrate", "Alpha_pinene": "Alpha-pinene"},
    "interval_min": 20.0,
    "interval_scale": 19.0,
    "odor_site_length": 50.0,
    "interval_normalization": 88.73,
    "prior_low": [-0.3, -1.3, -0.3, 0.0],
    "prior_high": [1.3, 1.3, 2, 0.3],
    "n_simulations": 2_000_000,
    "n_iter": 1000,
    "batch_size": 256,
    "n_early_stopping_patience": 5,
    "hidden_dim": 128,
    "num_layers": 8,
    "learning_rate": 1e-3,
    "transition_steps": 5000,
    "decay_rate": 0.999,
    "checkpoint_every": 20,  # Save checkpoint every 20 iterations
    "force_retrain": False,  # loads model that has already been trained if False, otherwise retrains from scratch even if model files exist
    "num_samples": 500,
    "num_warmup": 200,
    "num_chains": 2,
    "seed": 0,
}

param_names = ["drift_rate", "reward_bump", "failure_bump", "noise_std"]

print("Configuration loaded")
print(f"Model directory: {CONFIG['filename']}")

In [ ]:
# ============================================================================
# STEP 1: Initialize and Train
# ============================================================================

simulator = PatchForagingDDM_JAX(
    initial_prob=0.8,
    depletion_rate=-0.1,
    threshold=1.0,
    start_point=0.0,
    interval_min=CONFIG["interval_min"],
    interval_scale=CONFIG["interval_scale"],
    interval_normalization=CONFIG["interval_normalization"],
    odor_site_length=CONFIG["odor_site_length"],
    max_sites_per_window=CONFIG["window_size"],
    n_feat=CONFIG["n_feat"],
)

prior_fn = create_prior(prior_low=jnp.array(CONFIG["prior_low"]), prior_high=jnp.array(CONFIG["prior_high"]))

print("Simulator initialized")

# Create model directory and get checkpoint directory
model_dir, checkpoint_dir = get_model_directory(CONFIG)
model_path = model_dir / "model.pkl"

print(f"Model directory: {model_dir}")
print(f"Model file: {model_path}")

if model_path.exists() and not CONFIG["force_retrain"]:
    print(f"Loading existing model from {model_path}")
    with open(model_path, "rb") as f:
        model_data = pickle.load(f)
    snle = model_data["snle"]
    snle_params = model_data["snle_params"]
    y_mean = model_data["y_mean"]
    y_std = model_data["y_std"]
    print("✓ Model loaded successfully")
else:
    print("Training new SNLE model...")
    print(f"Config: {CONFIG}")

    rng_key = random.PRNGKey(CONFIG["seed"])

    snle, snle_params, losses, rng_key, y_mean, y_std = train_snle(
        simulator,
        prior_fn,
        mode="multi",
        n_simulations=CONFIG["n_simulations"],
        hidden_dim=CONFIG["hidden_dim"],
        num_layers=CONFIG["num_layers"],
        n_iter=CONFIG["n_iter"],
        batch_size=CONFIG["batch_size"],
        n_early_stopping_patience=CONFIG["n_early_stopping_patience"],
        learning_rate=CONFIG["learning_rate"],
        transition_steps=CONFIG["transition_steps"],
        decay_rate=CONFIG["decay_rate"],
        percentage_data_as_validation_set=0.1,
        rng_key=rng_key,
        save_dir=str(checkpoint_dir),
        checkpoint_every=CONFIG["checkpoint_every"],
    )

    # Save final model with descriptive filename
    print(f"\nSaving final model to {model_path}")
    model_data = {
        "snle_params": snle_params,
        "losses": losses,
        "y_mean": y_mean,
        "y_std": y_std,
        "config": CONFIG,
        "model_dir": str(model_dir),
        "checkpoint_dir": str(checkpoint_dir),
    }

with open(model_path, "wb") as f:
    pickle.dump(model_data, f)

    print(f"✓ Model saved successfully to {model_path}")
    print(f"✓ Checkpoints saved in {checkpoint_dir}")

print("\n" + "=" * 80)
print("Training complete!")
print(f"Model directory: {model_dir}")
print("=" * 80)

In [ ]:
# Save with descriptive filename
print(f"\nSaving model to {model_path}")
model_data = {
    "snle_params": snle_params,
    "losses": losses,
    "y_mean": y_mean,
    "y_std": y_std,
    "config": CONFIG,
}

with open(model_path, "wb") as f:
    pickle.dump(model_data, f)

print("✓ Model saved successfully")

In [ ]:
model_path = "/Users/laura.driscoll/Documents/code/sbi_results/snle_2M_lr0.0004_ts2000_h128_l8_b128_37feat/model.pkl"

with open(model_path, "rb") as f:
    model_data = pickle.load(f)

# Disable LaTeX rendering in matplotlib
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "sans-serif"

_, axes = plt.subplots(figsize=(6, 3))
plot_loss_profile(model_data["losses"], axes)
axes.set_ylim([-300, 100])
plt.show()

In [ ]:
CONFIG = model_data["config"]
with open(model_path, "rb") as f:
    model_data = pickle.load(f)

# Reconstruct SNLE
simulator = PatchForagingDDM_JAX(max_sites_per_window=100, n_feat=CONFIG["n_feat"])
prior_fn = create_prior(prior_low=jnp.array(CONFIG["prior_low"]), prior_high=jnp.array(CONFIG["prior_high"]))

rng_key = random.PRNGKey(CONFIG["seed"])
rng_key, test_key = random.split(rng_key)
test_theta = prior_fn().sample(seed=test_key)
test_x = simulator.simulator_fn(seed=test_key, theta=test_theta)
n_features = test_x.shape[-1]

flow = make_maf(
    n_dimension=n_features,  # obs data dimension
    n_layers=CONFIG["num_layers"],
    hidden_sizes=(CONFIG["hidden_dim"], CONFIG["hidden_dim"]),
)

snle = NLE((prior_fn, simulator.simulator_fn), flow)

print("✓ Model loaded!")

In [ ]:
# --- Simulate observed data ---
print("\n2. Simulating observed data...")
rng_key, subkey = random.split(rng_key)
true_theta = prior_fn().sample(seed=subkey)["theta"]
rng_key, subkey = random.split(rng_key)
_, observed_stats = simulator.simulate_one_window(true_theta, subkey)
print(f"   True theta: {true_theta}")
print(f"   Observed stats: {observed_stats}")

In [ ]:
# --- Run inference ---
print("\n3. Testing inference...")
rng_key, subkey = random.split(rng_key)
posterior_samples, diagnostics = infer_parameters_snle(
    snle,
    model_data["snle_params"],
    observed_stats,
    model_data["y_mean"],
    model_data["y_std"],
    num_samples=1_000,
    num_warmup=500,
    num_chains=4,
    rng_key=subkey,
)

In [ ]:
# --- Plot posterior distributions ---
param_names = ["drift_rate", "reward_bump", "failure_bump", "noise_std"]
param_labels = [
    "drift_rate: evidence accumulation rate",
    "reward_bump: evidence drop from receiving reward",
    "failure_bump: evidence boost from not receiving reward",
    "noise_std: std of noise in evidence accumulation",
]

fig, axes = plt.subplots(1, 4, figsize=(10, 2))
axes = axes.flatten()

for i in range(4):
    # Compute histogram
    counts, bins, _ = axes[i].hist(posterior_samples[:, i], bins=30, color="dodgerblue", edgecolor=None, alpha=0.7)

    # Posterior mode (bin center with max count)
    mode_index = jnp.argmax(counts)
    posterior_mode = (bins[mode_index] + bins[mode_index + 1]) / 2

    axes[i].axvline(true_theta[i], color="orangered", linestyle="--", label="true value")
    axes[i].axvline(posterior_mode, color="k", linestyle="--", label="MAP estimate")

    axes[i].set_xlabel(param_names[i])
    axes[i].set_xlim(CONFIG["prior_low"][i], CONFIG["prior_high"][i])
    axes[i].set_ylabel("Frequency")

axes[i].legend(loc="center left", bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

In [ ]:
pairplot(posterior_samples, true_theta, param_names, figsize_per_param=2.0)

In [ ]:
# Validation
results = validate_parameter_recovery(
    snle, snle_params, y_mean, y_std, simulator, prior_fn, infer_parameters_snle, n_tests=10
)
plot_recovery_scatter(results)

# Full SBC
ranks, z_scores = run_sbc_evaluation(
    snle,
    snle_params,
    y_mean,
    y_std,
    simulator,
    prior_fn,
    infer_parameters_snle,
    n_tests=100,
    save_path="sbc_results.png",
)

In [ ]:
# Generate multiple patches from posterior samples to compare distributions
print("\nGenerating patches from posterior samples for comparison...")

num_window_sites = 500

# --- Simulate 'real' data from simulator---
print("\n2. Simulating observed data...")
real_data = []
for i_site in range(num_window_sites):
    rng_key, subkey = random.split(rng_key)
    _, data = simulator.simulate_one_window(true_theta, subkey)
    real_data.append(data)
real_data = np.array(real_data)

# # 2. Generate "synthetic" data from simulator using posterior-sampled parameters
# Get observed summary stats
rng_key, subkey = random.split(rng_key)
_, observed_stats = simulator.simulate_one_window(true_theta, subkey)
obs_norm = (observed_stats - y_mean) / y_std

# Sample posterior θ using ONE observation
rng_key, sub = random.split(rng_key)
inference_results, diagnostics = snle.sample_posterior(
    sub,
    snle_params,
    observable=obs_norm,
    n_samples=500,
    n_chains=1,
    n_warmup=200,
)

posterior_ds = extract_samples(inference_results)
theta_samples = posterior_ds["theta"].values.reshape(-1, 4)  # shape (500,4)

# Simulate synthetic patches for comparison
posterior_mean_theta = theta_samples.mean(axis=0)  # Single point estimate

synthetic_data = []
for i in range(500):
    rng_key, subkey = random.split(rng_key)
    _, data = simulator.simulate_one_window(posterior_mean_theta, subkey)
    synthetic_data.append(data)
synthetic_data = jnp.array(synthetic_data)

# 4. Plot
plot_real_synth_hist(real_data, synthetic_data)